In [1]:
import json
import numpy as np
import wowsims

# Load Fury Raid Sim Request JSON

In [2]:
f = open('data/fury-human-bis-p3.json')
settings = json.load(f)

# Set environment and settings

In [3]:
def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))

In [4]:
# Iterations are currently capped at 3000
iterations = settings['simOptions']['iterations']

duration = settings['encounter']['duration']

In [5]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 11782.084625708363


Interactive mode is a way to provide input to the sim while it is running.
This can be used, for example, as a way to test rotation ideas.
Because interactive mode is enabled, the code below does nothing other than auto attack.

In [6]:
settings['simOptions']['interactive'] = True
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 3883.4096813369024


The next run uses the shred spell when it can.
Spells are accesed via the casting character's spellbook.
In this case, we have a help file `feral.py` that defines some globals for the spells we are interested in.

In [7]:
from fury import Spells

Spells.register()

settings['simOptions']['interactive'] = True
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        if wowsims.needsInput():
            wowsims.trySpell(Spells.Bloodthirst)
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 6270.832346789034


For more complex input logic having some context would be helpful.
The `feral.py` file also contains some aura labels and registers them with the sim. 

In [8]:
from fury import Auras, TargetAuras

Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        if wowsims.needsInput():
            Auras.update()
            TargetAuras.update()
            rage = wowsims.getRage()
            if rage >= 20:
                wowsims.trySpell(Spells.Bloodthirst)

    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 6256.112516090608


The examples so far have only looked at damage output.
More detailed spell metrics are also available.

In [9]:
settings['simOptions']['interactive'] = False

while not wowsims.step():
    pass
json_string = wowsims.getSpellMetrics()
cast_metrics = json.loads(json_string)
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    print(f'{spell_id}: {metrics[0]}')

0: {'Casts': 1, 'Misses': 0, 'Hits': 0, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 0, 'TotalThreat': 0, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
12867: {'Casts': 85, 'Misses': 0, 'Hits': 85, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 162215.27517715053, 'TotalThreat': 113550.69262400537, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
23881: {'Casts': 29, 'Misses': 0, 'Hits': 9, 'Crits': 20, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 252000.70577760728, 'TotalThreat': 176400.49404432514, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 43500000000}
